#Install

In [ ]:
!pip install python-pptx

from google.colab import drive
import os
import pptx
from pptx.enum.shapes import MSO_SHAPE_TYPE

!pip install openai
import openai
import os
import time
import pandas as pd
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 3.4 MB/s eta 0:00:00
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470949 sha256=3d42c28fccda0b8355a776b903fb0c124f1441de294c5276cc5afb87285dd487
  Stored in directory: /root/.cache/pip/wheels/0e/4a/ed/9653bc799915f52dce3f04d14946fbd85cce9c3cdedc9cfa71
Successfully built python-pptx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB

In [ ]:
#your API key
os.environ["OPENAI_API_KEY"] = '[YOUR_API_KEY]'

#load ppt from the google drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/analyzer/input_ppt")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Functions

In [ ]:
class ChatApp: #no demo, only template
    def __init__(self, input):
        # Setting the API key to use the OpenAI API
        openai.api_key = os.getenv("OPENAI_API_KEY")
        self.messages = input

    def chat(self, message):
        self.messages.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self.messages,
            temperature=0.7,
            top_p = 1,
        )
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})

        for i in range(2):
          self.messages.pop()

        return response["choices"][0]["message"]["content"]

In [ ]:
def make_prompt_format(system, user, assistant): #prompt demonstration
  #system
  input = [{"role": "system", "content": system}]
  if user != "":
    #user
    demo_user = []
    demo_user.append(user)
    #assistant
    demo_assistant = []
    demo_assistant.append(assistant)

    #make format for API
    demo = pd.DataFrame({'user': demo_user, 'assistant': demo_assistant})
    for prompt, completion in zip(demo['user'], demo['assistant']):
      message_dict = {"role": "user", "content": prompt}
      input.append(message_dict)
      message_dict = {"role": "assistant", "content": completion}
      input.append(message_dict)
    
  return input

#Main

##Load Data

In [ ]:
#load ppt file
ppt = pptx.Presentation('Architecture_Design.pptx')

#for full slide
full_slide = []
content_full = "Entire contents: "
for i, slide in enumerate(ppt.slides):
  content_full = content_full + f"  *slide {i}: "
  for shape in slide.shapes:
    if hasattr(shape, "text"):
      content_full = content_full + shape.text
full_slide = content_full

# print("*"*10)
# print(full_slide)

#for each slide, keywords
each_slide = {} #dictionary form
for i, slide in enumerate(ppt.slides):
  content_each = ""
  for shape in slide.shapes:
    if hasattr(shape, "text"):
      content_each = content_each + shape.text
  each_slide[f'{i}'] = content_each 

# print("*"*10)
# print(each_slide)

##Full Slide

In [ ]:
# Summarize the entire Powerpoint file.

In [ ]:
#prompt demonstration
full_system = "I want you to summarize the given content. I will provide you with the entire contents of a PowerPoint presentation, and you will provide me with a summary of the entire slides."
full_user = ""
full_assistant = ""
prompt_full = make_prompt_format(full_system, full_user, full_assistant)

#summarize full slide
app_full = ChatApp(prompt_full)
message = full_slide
result_full = app_full.chat(message)
time.sleep(3) #API call restriction, 20 calls per a minute

#print result
result_full

'Architectural design is a critical phase that identifies and organizes the main structural components of a system. The output is an architectural model that describes how the system is organized as a set of communicating components. The architecture of a software system affects performance, efficiency, security, and maintainability. Design flaws can result in project failure. Architectural design objectives include maintainability, use of commercial off-the-shelf parts, system performance, security, safety, availability, reliability, fault tolerance, and recovery. Software design is a two-step process that includes architectural design and detailed design. There are four types of systems/subsystems: interactive, event-driven, transformational, and database. Popular software architecture styles include client-server, n-tier or multi-tier, main program and subroutines, event-driven, and persistence framework. The architectural design process involves selecting the appropriate style for 